# Plot the Near-Inertial Activity Forecast

To see how we got here, check the notebooks
- [loading the data](./01_download_GFS_data.html)
- and [running the slab-ocean model](./02_run_slab_model.html).

Details: https://willirath.github.io/nia-prediction-low-latitutdes

## Parameters

In [ ]:
# parameters

GFS_zarr_store = "tmp_GFS.zarr"
slab_zarr_store = "tmp_slab.zarr"

dask_kwargs = {"n_workers": 1, "threads_per_worker": 3, "memory_limit": 6e9}

## Tech preamble

Import modules and spin up Dask cluster.

In [ ]:
import xarray as xr
from dask.distributed import Client
import hvplot.xarray

The memory requirements of the integration are currently relatively high. So we'll go for one single-threaded Worker with enough memory.

If we'll do this with Github Actions, there will be a total of [6GB memory](https://docs.github.com/en/actions/reference/virtual-environments-for-github-hosted-runners#supported-runners-and-hardware-resources) for all we do. So let's stay below this even in development.

In [ ]:
client = Client(**dask_kwargs)
client

## Loading the data

In [ ]:
ds_GFS = xr.open_zarr(GFS_zarr_store)
ds_slab = xr.open_zarr(slab_zarr_store)

In [ ]:
import holoviews as hv

In [ ]:
start_of_forecast = (~ds_GFS["is_forecast"].astype(bool)).isel(lat=0, lon=0, drop=True).sum().compute().data
start_of_forecast = ds_GFS["time"].data[start_of_forecast]

In [ ]:
(
    hv.VLine(start_of_forecast)
    * ds_GFS["taux"].sel(lon=360-23, lat=12, method="nearest").hvplot.line(label="taux")
    * ds_GFS["tauy"].sel(lon=360-23, lat=12, method="nearest").hvplot.line(label="tauy")
    + hv.VLine(start_of_forecast)
    * ds_slab["u_slab"].sel(lon=360-23, lat=12, method="nearest").hvplot.line(label="u")
    * ds_slab["v_slab"].sel(lon=360-23, lat=12, method="nearest").hvplot.line(label="v")
    * ds_slab["umag_slab"].sel(lon=360-23, lat=12, method="nearest").hvplot.line(label="umag")
).cols(1)

In [ ]:
import numpy as np

In [ ]:
ds_slab["umag_slab"].max(
    "time"
).hvplot.contourf(
    levels=list(np.linspace(0, 2, 20))
)

In [ ]:
!echo "Finished: $(date -Ins)"

---
See https://github.com/willirath/nia-prediction-low-latitutdes for details.